langchain-core: The base abstractions (like runnable, messages, and standard interfaces).

langchain: The chain, agent, and retrieval logic.

langchain-community: The massive collection of specific tools, document loaders, vector stores, and chat models provided by the community and third parties (e.g., Wikipedia tools, FAISS vector store, specific PDF loaders).

We will be taking an article draft and using LangChain to generate various useful items around this article. We'll be creating:

01. An article title
02. An article description
03. Editor advice where we will insert an additional paragraph in the article
04. A thumbnail / hero image for our article.

Here we input our article to start with. Currently this is using an article from the Aurelio AI learning page.


In [48]:
import os
from langchain_openai import ChatOpenAI
import textwrap

os.environ["OPENAI_API_KEY"] = "none"
# MODEL = "functiongemma:270m" # still no good
MODEL = "qwen3:0.6b" # okay
# MODEL = "gemma3:4b" # okay
BASE_URL = "http://localhost:11434/v1"

In [49]:
llm = ChatOpenAI(model=MODEL, base_url=BASE_URL, temperature=0.0)

creative_llm = ChatOpenAI(model=MODEL, base_url=BASE_URL, temperature=0.9)

# response = model.invoke("Why do parrots talk?")
# response.content

In [50]:
article = """
\
We believe AI's short—to mid-term future belongs to agents and that the long-term future of *AGI* may evolve from agentic systems. Our definition of agents covers any neuro-symbolic system in which we merge neural AI (such as an LLM) with semi-traditional software.

With agents, we allow LLMs to integrate with code — allowing AI to search the web, perform math, and essentially integrate into anything we can build with code. It should be clear the scope of use cases is phenomenal where AI can integrate with the broader world of software.

In this introduction to AI agents, we will cover the essential concepts that make them what they are and why that will make them the core of real-world AI in the years to come.

---

## Neuro-Symbolic Systems

Neuro-symbolic systems consist of both neural and symbolic computation, where:

- Neural refers to LLMs, embedding models, or other neural network-based models.
- Symbolic refers to logic containing symbolic logic, such as code.

Both neural and symbolic AI originate from the early philosophical approaches to AI: connectionism (now neural) and symbolism. Symbolic AI is the more traditional AI. Diehard symbolists believed they could achieve true AGI via written rules, ontologies, and other logical functions.

The other camp were the connectionists. Connectionism emerged in 1943 with a theoretical neural circuit but truly kicked off with Rosenblatt's perceptron paper in 1958 [1][2]. Both of these approaches to AI are fascinating but deserve more time than we can give them here, so we will leave further exploration of these concepts for a future chapter.

Most important to us is understanding where symbolic logic outperforms neural-based compute and vice-versa.

| Neural | Symbolic |
| --- | --- |
| Flexible, learned logic that can cover a huge range of potential scenarios. | Mostly hand-written rules which can be very granular and fine-tuned but hard to scale. |
| Hard to interpret why a neural system does what it does. Very difficult or even impossible to predict behavior. | Rules are written and can be understood. When unsure why a particular ouput was produced we can look at the rules / logic to understand. |
| Requires huge amount of data and compute to train state-of-the-art neural models, making it hard to add new abilities or update with new information. | Code is relatively cheap to write, it can be updated with new features easily, and latest information can often be added often instantaneously. |
| When trained on broad datasets can often lack performance when exposed to unique scenarios that are not well represented in the training data. | Easily customized to unique scenarios. |
| Struggles with complex computations such as mathematical operations. | Perform complex computations very quickly and accurately. |

Pure neural architectures struggle with many seemingly simple tasks. For example, an LLM *cannot* provide an accurate answer if we ask it for today's date.

Retrieval Augmented Generation (RAG) is commonly used to provide LLMs with up-to-date knowledge on a particular subject or access to proprietary knowledge.

### Giving LLMs Superpowers

By 2020, it was becoming clear that neural AI systems could not perform tasks symbolic systems typically excelled in, such as arithmetic, accessing structured DB data, or making API calls. These tasks require discrete input parameters that allow us to process them reliably according to strict written logic.

In 2022, researchers at AI21 developed Jurassic-X, an LLM-based "neuro-symbolic architecture." Neuro-symbolic refers to merging the "neural computation" of large language models (LLMs) with more traditional (i.e. symbolic) computation of code.

Jurassic-X used the Modular Reasoning, Knowledge, and Language (MRKL) system [3]. The researchers developed MRKL to solve the limitations of LLMs, namely:

- Lack of up-to-date knowledge, whether that is the latest in AI or something as simple as today's date.
- Lack of proprietary knowledge, such as internal company docs or your calendar bookings.
- Lack of reasoning, i.e. the inability to perform operations that traditional software is good at, like running complex mathematical operations.
- Lack of ability to generalize. Back in 2022, most LLMs had to be fine-tuned to perform well in a specific domain. This problem is still present today but far less prominent as the SotA models generalize much better and, in the case of MRKL, are able to use tools relatively well (although we could certainly take the MRKL solution to improve tool use performance even today).

MRKL represents one of the earliest forms of what we would now call an agent; it is an LLM (neural computation) paired with executable code (symbolic computation).

## ReAct and Tools

There is a misconception in the broader industry that an AI agent is an LLM contained within some looping logic that can generate inputs for and execute code functions. This definition of agents originates from the huge popularity of the ReAct agent framework and the adoption of a similar structure with function/tool calling by LLM providers such as OpenAI, Anthropic, and Ollama.

![ReAct agent flow with the Reasoning-Action loop [4]. When the action chosen specifies to use a normal tool, the tool is used and the observation returned for another iteration through the Reasoning-Action loop. To return a final answer to the user the LLM must choose action "answer" and provide the natural language response, finishing the loop.](/images/posts/ai-agents/ai-agents-00.png)

<small>ReAct agent flow with the Reasoning-Action loop [4]. When the action chosen specifies to use a normal tool, the tool is used and the observation returned for another iteration through the Reasoning-Action loop. To return a final answer to the user the LLM must choose action "answer" and provide the natural language response, finishing the loop.</small>

Our "neuro-symbolic" definition is much broader but certainly does include ReAct agents and LLMs paired with tools. This agent type is the most common for now, so it's worth understanding the basic concept behind it.

The **Re**ason **Act**ion (ReAct) method encourages LLMs to generate iterative *reasoning* and *action* steps. During *reasoning,* the LLM describes what steps are to be taken to answer the user's query. Then, the LLM generates an *action,* which we parse into an input to some executable code, which we typically describe as a tool/function call.

![ReAct method. Each iteration includes a Reasoning step followed by an Action (tool call) step. The Observation is the output from the previous tool call. During the final iteration the agent calls the answer tool, meaning we generate the final answer for the user.](/images/posts/ai-agents/ai-agents-01.png)

<small>ReAct method. Each iteration includes a Reasoning step followed by an Action (tool call) step. The Observation is the output from the previous tool call. During the final iteration the agent calls the answer tool, meaning we generate the final answer for the user.</small>

Following the reason and action steps, our action tool call returns an observation. The logic returns the observation to the LLM, which is then used to generate subsequent reasoning and action steps.

The ReAct loop continues until the LLM has enough information to answer the original input. Once the LLM reaches this state, it calls a special *answer* action with the generated answer for the user.

## Not only LLMs and Tool Calls

LLMs paired with tool calling are powerful but far from the only approach to building agents. Using the definition of neuro-symbolic, we cover architectures such as:

- Multi-agent workflows that involve multiple LLM-tool (or other agent structure) combinations.
- More deterministic workflows where we may have set neural model-tool paths that may fork or merge as the use case requires.
- Embedding models that can detect user intents and decide tool-use or LLM selection-based selection in vector space.

These are just a few high-level examples of alternative agent structures. Far from being designed for niche use cases, we find these alternative options to frequently perform better than the more common ReAct or Tool agents. We will cover all of these examples and more in future chapters.

---

Agents are fundamental to the future of AI, but that doesn't mean we should expect that future to come from agents in their most popular form today. ReAct and Tool agents are great and handle many simple use cases well, but the scope of agents is much broader, and we believe thinking beyond ReAct and Tools is key to building future AI.

---

You can sign up for the [Aurelio AI newsletter](https://b0fcw9ec53w.typeform.com/to/w2BDHVK7) to stay updated on future releases in our comprehensive course on agents.

---

## References

[1] The curious case of Connectionism (2019) [https://www.degruyter.com/document/doi/10.1515/opphil-2019-0018/html](https://www.degruyter.com/document/doi/10.1515/opphil-2019-0018/html)

[2] F. Rosenblatt, [The Perceptron: A Probabilistic Model for Information Storage and Organization in the Brain](https://www.ling.upenn.edu/courses/cogs501/Rosenblatt1958.pdf) (1958), Psychological Review

[3] E. Karpas et al. [MRKL Systems: A Modular, Neuro-Symbolic Architecture That Combines Large Language Models, External Knowledge Sources and Discrete Reasoning](https://arxiv.org/abs/2205.00445) (2022), AI21 Labs
"""

In [51]:
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant called {name} that helps generate article titles.",
    input_variable=["name"],
)

user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a name for a article.
The article is here for you to examine:

---

 {article}
 
---
The name should be based of the context of the article.
Be creative, but make sure the names are clear, catchy,
and relevant to the theme of the article.

Only output the article name, no other explanation or
text can be provided and need it simple and short.""",
    input_variables=["article"],
)

In [52]:
# user_prompt.format(article="TEST STRING")

In [53]:
# preview of the user prompt output. Later for the variable `article` we can give a real value.
# print(user_prompt.format(article="TEST STRING").content)

In [54]:
from langchain_core.prompts import ChatPromptTemplate

first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

In [55]:
# format `first_prompt` also

first_prompt.format(article="TEST STRING", name="TEST NAME")

'System: You are an AI assistant called TEST NAME that helps generate article titles.\nHuman: You are tasked with creating a name for a article.\nThe article is here for you to examine:\n\n---\n\n TEST STRING\n\n---\nThe name should be based of the context of the article.\nBe creative, but make sure the names are clear, catchy,\nand relevant to the theme of the article.\n\nOnly output the article name, no other explanation or\ntext can be provided and need it simple and short.'

In [56]:
print(first_prompt.format(article="TEST STRING", name="TEST NAME"))

System: You are an AI assistant called TEST NAME that helps generate article titles.
Human: You are tasked with creating a name for a article.
The article is here for you to examine:

---

 TEST STRING

---
The name should be based of the context of the article.
Be creative, but make sure the names are clear, catchy,
and relevant to the theme of the article.

Only output the article name, no other explanation or
text can be provided and need it simple and short.


In [ ]:
chain_one = (
    {
        "article": lambda x: x["article"],
        "name": lambda x: x["name"],
    }
    | first_prompt
    | creative_llm
    | {"article_title": lambda x: x.content} # extract the content field of llm output
)



# {left side} | {right side} => left is going to go into the right side
# {...} go into `first_prompt`, first prompt go into `creative_llm`, creative llm go into {...}

article_title_msg = chain_one.invoke({
    "article" : article,
    "name" : "Title Finder"
})

article_title_msg

In [ ]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that helps build good articles.",
)

second_user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a description for
the article. The article is here for you to examine:

---

{article}

---

Here is the article title '{article_title}'.

Output the SEO friendly article description. 
make sure we don't exceed 120 characters.
Do not output anything other than the description.""",
    input_variables=["article", "article_title"],
)

second_prompt = ChatPromptTemplate.from_messages([system_prompt, second_user_prompt])


chain_two = (
    {
        "article": lambda x: x["article"], 
        "article_title": lambda x: x["article_title"]
    }
    | second_prompt
    | llm
    | {"summery": lambda x: x.content}
)

article_description_msg = chain_two.invoke (
    {
        "article": article,
        "article_title": article_title_msg["article_title"]
    }
)

article_description_msg

{'summery': 'Neuro-symbolic agents for the future of AI: explore how combining neural and symbolic systems will shape the next generation of AI. With applications in search, computation, and integration, these agents will redefine the boundaries of real-world AI. The MRKL architecture, ReAct method, and future use cases are highlighted. [1-3]'}

In [ ]:
third_user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a new paragraph for the
article. The article is here for you to examine:

---

{article}

---

Choose one paragraph to review and edit. During your edit
ensure you provide constructive feedback to the user so they
can learn where to improve their own writing.""",
    input_variables=["article"]
)

# prompt template 3: creating a new paragraph for the article
third_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    third_user_prompt
])


#* force a Large Language Model (LLM) to output data in a strict, structured format rather than free text.
from pydantic import BaseModel, Field

class Paragraph(BaseModel):
    original_paragraph: str = Field(description="The original paragraph")
    edited_paragraph: str = Field(description="The improved edited paragraph")
    feedback: str = Field(
        description = "Constructive feedback on the original paragraph."
    )

structured_llm = creative_llm.with_structured_output(Paragraph)


chain_tree = (
    {"article": lambda x: x["article"]}
    | third_prompt
    | structured_llm
    | {
        "original_paragraph": lambda x: x.original_paragraph,
        "edited_paragraph": lambda x: x.edited_paragraph,
        "feedback": lambda x: x.feedback
    }
)

out = chain_tree.invoke({"article": article})
out

{'original_paragraph': "We believe AI's short—to mid-term future belongs to agents and that the long-term future of *AGI* may evolve from agentic systems. Our definition of agents covers any neuro-symbolic system in which we merge neural AI (such as an LLM) with semi-traditional software.",
 'edited_paragraph': "We believe AI's short-term to the mid-term future belongs to agents, and that the long-term future of *AGI* may evolve through agentic systems. Our definition of agents includes any neuro-symbolic systems that integrate neural AI (like large language models) with traditional software. These systems enable AI to perform a wide array of tasks, from conducting complex computations to interacting with external knowledge bases.",
 'feedback': "The paragraph highlights the importance of agents for the future, making it a strong start. You can expand on the term 'short-term' to better emphasize the focus on current and future systems. The definition could be more precise, emphasizing 

## image generation

use a model that can generate images (like open ai dall-e-3)

In [ ]:
# from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
# from langchain_core.prompts import PromptTemplate

# image_prompt = PromptTemplate(
#     input_variables=["article"],
#     template=(
#         "Generate a prompt with less then 500 characters to generate an image "
#         "based on the following article: {article}"
#     ),
# )


# from skimage import io
# import matplotlib.pyplot as plt
# from langchain_core.runnables import RunnableLambda


# def generate_and_display_image(image_prompt):
#     image_url = DallEAPIWrapper(model="dall-e-3").run(image_prompt)
#     image_data = io.imread(image_url)

#     # And update the display code to:
#     plt.imshow(image_data)
#     plt.axis("off")
#     plt.show()


# # we wrap this in a RunnableLambda for use with LCEL
# image_gen_runnable = RunnableLambda(generate_and_display_image)


# # chain 4: inputs: article, article_para / outputs: new_suggestion_article
# chain_four = (
#     {"article": lambda x: x["article"]}
#     | image_prompt
#     | llm
#     | (lambda x: x.content)
#     | image_gen_runnable
# )


# chain_four.invoke({"article": article})

# Langsmith

---

In [ ]:
import os
from getpass import getpass

# must enter API key
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or getpass(
    "Enter LangSmith API Key: "
)

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "langchain_course_1"

In [ ]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "none"
# MODEL = "functiongemma:270m" # still no good
MODEL = "qwen3:0.6b"  # okay
# MODEL = "gemma3:4b" # okay
BASE_URL = "http://localhost:11434/v1"

In [ ]:
# testing

import os
from getpass import getpass
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

llm = ChatOpenAI(temperature=0.0, model=MODEL, base_url=BASE_URL)

llm.invoke("hello")

AIMessage(content='Hello! How can I assist you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 11, 'total_tokens': 104, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'qwen3:0.6b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-150', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b5b26-b02c-76f3-8324-fae4de46a2ba-0', usage_metadata={'input_tokens': 11, 'output_tokens': 93, 'total_tokens': 104, 'input_token_details': {}, 'output_token_details': {}})

### Tracing Non-LangChain Code

In [ ]:
from langsmith import traceable
import random
import time


@traceable
def generate_random_number():
    return random.randint(0, 100)


@traceable
def generate_string_delay(input_str: str):
    number = random.randint(1, 5)
    time.sleep(number)
    return f"{input_str} ({number})"


@traceable
def random_error():
    number = random.randint(0, 1)
    if number == 0:
        raise ValueError("Random error")
    else:
        return "No error"

In [ ]:
from tqdm.auto import tqdm

for _ in tqdm(range(10)):
    generate_random_number()
    generate_string_delay("Hello")
    try:
        random_error()
    except ValueError:
        pass

  0%|          | 0/10 [00:00<?, ?it/s]

#### with specific names

In [ ]:
from langsmith import traceable


@traceable(name="Chitchat Maker")
def error_generation_function(question: str):
    delay = random.randint(0, 3)
    time.sleep(delay)
    number = random.randint(0, 1)
    if number == 0:
        raise ValueError("Random error")
    else:
        return "I'm great how are you?"

In [ ]:
for _ in tqdm(range(5)):
    try:
        error_generation_function("How are you today?")
    except ValueError:
        pass

  0%|          | 0/5 [00:00<?, ?it/s]

---

# Prompting technique

We'll start by looking at the various parts of our prompt. For RAG use-cases we'll typically have three core components however this is _very_ use-cases dependant and can vary significantly. Nonetheless, for RAG we will typically see:

* **Rules for our LLM**: this part of the prompt sets up the behavior of our LLM, how it should approach responding to user queries, and simply providing as much information as possible about what we're wanting to do as possible. We typically place this within the _system prompt_ of an chat LLM.

* **Context**: this part is RAG-specific. The context refers to some _external information_ that we may have retrieved from a web search, database query, or often a _vector database_. This external information is the **R**etrieval **A**ugmentation part of **RA**G. For chat LLMs we'll typically place this inside the chat messages between the assistant and user.

* **Question**: this is the input from our user. In the vast majority of cases the question/query/user input will always be provided to the LLM (and typically through a _user message_). However, the format and location of this being provided often changes.

* **Answer**: this is the answer from our assistant, again this is _very_ typical and we'd expect this with every use-case.

The below is an example of how a RAG prompt may look:

```
Answer the question based on the context below,                 }
if you cannot answer the question using the                     }--->  (Rules) For Our Prompt
provided information answer with "I don't know"                 }

Context: Aurelio AI is an AI development studio                 }
focused on the fields of Natural Language Processing (NLP)      }
and information retrieval using modern tooling                  }--->   Context AI has
such as Large Language Models (LLMs),                           }
vector databases, and LangChain.                                }

Question: Does Aurelio AI do anything related to LangChain?     }--->   User Question

Answer:                                                         }--->   AI Answer
```

In [ ]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "none"
# MODEL = "functiongemma:270m" # still no good
MODEL = "qwen3:0.6b"  # okay
# MODEL = "gemma3:4b" # okay
BASE_URL = "http://localhost:11434/v1"

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Context: {context}
"""


# from langchain.prompts import (
#     SystemMessagePromptTemplate,
#     HumanMessagePromptTemplate
# )

# prompt_template = ChatPromptTemplate.from_messages([
#     SystemMessagePromptTemplate.from_template(prompt),
#     HumanMessagePromptTemplate.from_template("{query}"),
# ])

# short way
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", prompt),
        ("user", "{query}"), # now langchain can automatically detects inputs. so no need to specify input_variables.
    ]
)

In [ ]:
prompt_template.input_variables

['context', 'query']

In [ ]:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=MODEL, base_url=BASE_URL, temperature=0.0)

# pipeline = (
#     {
#         "query": lambda x: x["query"], # to these can add (x["query"].lower())
#         "context": lambda x: x["context"]
#     }
#     | prompt_template
#     | llm
# )

pipeline = prompt_template | llm #can use like this also

context = """Aurelio AI is an AI company developing tooling for AI
engineers. Their focus is on language AI with the team having strong
expertise in building AI agents and a strong background in
information retrieval.

The company is behind several open source frameworks, most notably
Semantic Router and Semantic Chunkers. They also have an AI
Platform providing engineers with tooling to help them build with
AI. Finally, the team also provides development services to other
organizations to help them bring their AI tech to market.

Aurelio AI became LangChain Experts in September 2024 after a long
track record of delivering AI solutions built with the LangChain
ecosystem."""

query = "what does Aurelio AI do?"

In [ ]:
prompt_template.format( query = "___QUERY___", context = "___CONTEXT___")

'System: \nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: ___CONTEXT___\n\nHuman: ___QUERY___'

In [ ]:
response = pipeline.invoke({"query": query, "context": context})
response.content

'Aurelio AI develops tooling for AI engineers, focuses on language AI with expertise in building AI agents and information retrieval, provides open-source frameworks like Semantic Router and Semantic Chunkers, an AI Platform for building AI tools, and offers development services to other organizations. They became LangChain Experts in September 2024 after a long track record of delivering AI solutions built with the LangChain ecosystem.'

`video time : 1:12:40`

### Few Shot Prompting

In [ ]:
# template
from langchain_core.prompts import FewShotChatMessagePromptTemplate


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"), 
        ("ai", "{output}")
    ]
)


examples = [
    {"input": "Here is query #1", "output": "Here is the answer #1"},
    {"input": "Here is query #2", "output": "Here is the answer #2"},
    {"input": "Here is query #3", "output": "Here is the answer #3"},
]


few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt= example_prompt,
    examples = examples
)

few_shot_prompt.format()

'Human: Here is query #1\nAI: Here is the answer #1\nHuman: Here is query #2\nAI: Here is the answer #2\nHuman: Here is query #3\nAI: Here is the answer #3'

In [ ]:
# Few-Shot Example

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

new_system_prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Always answer in markdown format. When doing so please
provide headers, short summaries, follow with bullet
points, then conclude.

Context: {context}
"""

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", new_system_prompt),
        ("user", "{query}"),
    ]
)

llm = ChatOpenAI(model=MODEL, base_url=BASE_URL, temperature=0.5)


pipeline = prompt_template | llm

context = """Aurelio AI is an AI company developing tooling for AI
engineers. Their focus is on language AI with the team having strong
expertise in building AI agents and a strong background in
information retrieval.

The company is behind several open source frameworks, most notably
Semantic Router and Semantic Chunkers. They also have an AI
Platform providing engineers with tooling to help them build with
AI. Finally, the team also provides development services to other
organizations to help them bring their AI tech to market.

Aurelio AI became LangChain Experts in September 2024 after a long
track record of delivering AI solutions built with the LangChain
ecosystem."""

query = "what does Aurelio AI do?"

out = pipeline.invoke({"query": query, "context": context}).content

print(out)

- Develop tooling for AI engineers focused on language AI  
- Focus on building AI agents and information retrieval systems  
- Create open-source frameworks like Semantic Router and Semantic Chunkers  
- Offer an AI Platform for engineers to build with AI  
- Provide development services to other organizations  

Aurelio AI has a strong focus on AI engineering, open-source frameworks, and AI platform development, with a track record of delivering solutions built on the LangChain ecosystem. They became LangChain Experts in September 2024.


In [ ]:
from IPython.display import display, Markdown

display(Markdown(out))

- Develop tooling for AI engineers focused on language AI  
- Focus on building AI agents and information retrieval systems  
- Create open-source frameworks like Semantic Router and Semantic Chunkers  
- Offer an AI Platform for engineers to build with AI  
- Provide development services to other organizations  

Aurelio AI has a strong focus on AI engineering, open-source frameworks, and AI platform development, with a track record of delivering solutions built on the LangChain ecosystem. They became LangChain Experts in September 2024.

In [ ]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from IPython.display import display, Markdown
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate



examples = [
    {
        "input": "Can you explain gravity?",
        "output": (
            "\n## Gravity\n\n"
            "Gravity is one of the fundamental forces in the universe.\n\n"
            "### Discovery\n\n"
            "* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n"
            "* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n"
            "### In General Relativity\n\n"
            "* Gravity is described as the curvature of spacetime.\n"
            "* The more massive an object is, the more it curves spacetime.\n"
            "* This curvature is what causes objects to fall towards each other.\n\n"
            "### Gravitons\n\n"
            "* Gravitons are hypothetical particles that mediate the force of gravity.\n"
            "* They have not yet been detected.\n\n"
            "**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.\n\n"
        )
    },
    {
        "input": "What is the capital of France?",
        "output": (
            "\n## France\n\n"
            "The capital of France is Paris.\n\n"
            "### Origins\n\n"
            "* The name Paris comes from the Latin word \"Parisini\" which referred to a Celtic people living in the area.\n"
            "* The Romans named the city Lutetia, which means \"the place where the river turns\".\n"
            "* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.\n\n"
            "**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.\n\n"
        )
    }
]



example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"), 
        ("ai", "{output}")
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)



out = few_shot_prompt.format()

display(Markdown(out))



Human: Can you explain gravity?
AI: 
## Gravity

Gravity is one of the fundamental forces in the universe.

### Discovery

* Gravity was first discovered by Sir Isaac Newton in the late 17th century.
* It was said that Newton theorized about gravity after seeing an apple fall from a tree.

### In General Relativity

* Gravity is described as the curvature of spacetime.
* The more massive an object is, the more it curves spacetime.
* This curvature is what causes objects to fall towards each other.

### Gravitons

* Gravitons are hypothetical particles that mediate the force of gravity.
* They have not yet been detected.

**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.


Human: What is the capital of France?
AI: 
## France

The capital of France is Paris.

### Origins

* The name Paris comes from the Latin word "Parisini" which referred to a Celtic people living in the area.
* The Romans named the city Lutetia, which means "the place where the river turns".
* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.

**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.



In [ ]:
few_shot_prompt

FewShotChatMessagePromptTemplate(examples=[{'input': 'Can you explain gravity?', 'output': '\n## Gravity\n\nGravity is one of the fundamental forces in the universe.\n\n### Discovery\n\n* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n### In General Relativity\n\n* Gravity is described as the curvature of spacetime.\n* The more massive an object is, the more it curves spacetime.\n* This curvature is what causes objects to fall towards each other.\n\n### Gravitons\n\n* Gravitons are hypothetical particles that mediate the force of gravity.\n* They have not yet been detected.\n\n**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.\n\n'}, {'input': 'What is the capital of France?', 'output': '\n## France\n\nThe capital of France is Paris.\n\n### Origins\n\n* The name Paris comes from the Latin word "Parisini" which 

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", new_system_prompt),
    few_shot_prompt,
    ("user", "{query}"),
])

pipeline = prompt_template | llm


llm = ChatOpenAI(model=MODEL, base_url=BASE_URL)

out = pipeline.invoke({"query": query, "context": context}).content
display(Markdown(out))



## Aurelio AI: AI Tooling for Engineers

### Summary

Aurelio AI is an AI company focused on providing tools and services to AI engineers. They specialize in language AI, with a strong emphasis on building AI agents and leveraging information retrieval techniques.

### Key Activities

*   **Open Source Frameworks:** They develop and maintain open-source frameworks like Semantic Router and Semantic Chunkers, primarily for language AI applications.
*   **AI Platform:** They offer an AI Platform that equips engineers with tools to build AI solutions.
*   **Development Services:** Aurelio AI also provides development services to other organizations, assisting them in bringing their AI technology to market.
*   **LangChain Expertise:** Since September 2024, they've become LangChain Experts, signifying their significant experience in utilizing the LangChain ecosystem.

**To conclude**, Aurelio AI’s core business is to empower AI engineers with tools and expertise for developing advanced language AI solutions.

In [ ]:
# this is an extra code i just wanted to see the stream 

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", new_system_prompt),
        few_shot_prompt,
        ("user", "{query}"),
    ]
)

pipeline = prompt_template | llm


llm = ChatOpenAI(model=MODEL, base_url=BASE_URL)

# Create the stream generator
stream = pipeline.stream({"query": query, "context": context})

# You must iterate over the stream to get chunks
print("Streaming response:")
for chunk in stream:
    # In LangChain, 'chunk' is usually an AIMessageChunk object
    print(chunk.content, end="", flush=True)

Streaming response:
## What does Aurelio AI do?  

- **Focus on Language AI**: Develops AI agents and tools for language processing.  
- **Open-Source Frameworks**: Creates Semantic Router and Semantic Chunkers, which help with AI engineering.  
- **AI Platform**: Offers tooling for building AI systems.  
- **Development Services**: Provides support to other organizations bringing AI technology to market.  

**Conclusion**: Aurelio AI specializes in language AI innovation, framework development, and AI platform support.

In [ ]:
# with markdown

from IPython.display import display, Markdown

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", new_system_prompt),
        few_shot_prompt,
        ("user", "{query}"),
    ]
)

pipeline = prompt_template | llm

# --- The Streaming Logic ---

# 1. Create the stream generator
stream = pipeline.stream({"query": query, "context": context})

# 2. Create a placeholder in the output area
#    display_id=True allows us to reference this specific output later
display_handle = display(Markdown(""), display_id=True)

# 3. Initialize a variable to store the full text as it builds up
full_response = ""

for chunk in stream:
    # Check if content exists (sometimes chunks are empty)
    if chunk.content:
        # A. Append the new piece to our full text
        full_response += chunk.content
        
        # B. "Update" the specific display handle with the new full text
        #    This re-renders the Markdown every time a chunk arrives
        display_handle.update(Markdown(full_response))

Aurelio AI is an AI-driven company focused on language AI tools. Here are their key activities:  
- **Language and AI Agents**: Building AI agents and expertise in information retrieval.  
- **Open Source Tools**: Develops frameworks like Semantic Router and Semantic Chunkers.  
- **AI Platform**: Provides engineers with tooling for AI development.  
- **Services**: Offers development services to other organizations to integrate AI technologies.  

**To conclude**, Aurelio AI specializes in building AI systems for engineers and developers.

In [ ]:
no_cot_system_prompt = """
Be a helpful assistant and answer the user's question.

You MUST answer the question directly without any other
text or explanation.
"""

no_cot_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", no_cot_system_prompt),
        ("user", "{query}"),
    ]
)

query = "How many keystrokes are needed to type the numbers from 1 to 500?"
llm = ChatOpenAI(model=MODEL, base_url=BASE_URL)
no_cot_pipeline = no_cot_prompt_template | llm
no_cot_result = no_cot_pipeline.invoke({"query": query}).content
print(no_cot_result)

1392


In [ ]:
# Define the chain-of-thought prompt template
cot_system_prompt = """
Be a helpful assistant and answer the user's question.

To answer the question, you must:

- List systematically and in precise detail all
  sub problems that need to be solved to answer the
  question.
- Solve each sub problem INDIVIDUALLY and in sequence.
- Finally, use everything you have worked through to
  provide the final answer.
"""

cot_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", cot_system_prompt),
        ("user", "{query}"),
    ]
)

llm = ChatOpenAI(model=MODEL, base_url=BASE_URL)
cot_pipeline = cot_prompt_template | llm

cot_result = cot_pipeline.invoke({"query": query}).content
display(Markdown(cot_result))

To determine the total number of keystrokes needed to type numbers from 1 to 500, we break it down into separate digit ranges:

1. **Single-digit numbers (1 to 9):**  
   - 9 numbers (1–9) each have 1 digit.  
   - Keystrokes: $9 \times 1 = 9$.

2. **Two-digit numbers (10 to 99):**  
   - 90 numbers (10–99) each have 2 digits.  
   - Keystrokes: $90 \times 2 = 180$.

3. **Three-digit numbers (100 to 500):**  
   - 400 numbers (100–500) each have 3 digits.  
   - Keystrokes: $400 \times 3 = 1200$.

**Summing all:**  
$9 + 180 + 1200 = 1389$ keystrokes.

**Final Answer:** 1389.

In [ ]:
system_prompt = """
Be a helpful assistant and answer the user's question.
"""

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{query}"),
    ]
)

llm = ChatOpenAI(model=MODEL, base_url=BASE_URL )
pipeline = prompt_template | llm


result = pipeline.invoke({"query": query}).content
display(Markdown(result))

To calculate the total number of keystrokes needed to type numbers from 1 to 500, we break down the problem into digits in each position (units, tens, hundreds):

1. **Units place**:  
   - Each digit (0-9) appears for groups of 90 numbers (100-499).  
   - Total digits in units place: $90 + 90 + 1 = 181$.  

2. **Tens place**:  
   - Each digit (0-9) appears for groups of 90 numbers (10-99).  
   - Total digits in tens place: $90 + 90 + 90 = 270$.  

3. **Hundreds place**:  
   - Each digit (0-9) appears in hundreds place for numbers 100-500.  
   - Total digits in hundreds place: $100 + 100 + 100 = 300$.  

**Total keystrokes**: $181 + 270 + 300 = 751$.  

**Answer**: 751 keystrokes.

---

## Chat Memory

### 01. ConversationBufferMemory


ConversationBufferMemory is the simplest form of conversational memory, it is literally just a place that we store messages, and then use to feed messages into our LLM.

#### `Old Method`

In [1]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "none"
# MODEL = "functiongemma:270m" # still no good
# MODEL = "qwen3:0.6b"  # okay
MODEL = "gemma3:4b" # okay
BASE_URL = "http://localhost:11434/v1"

In [2]:
from langchain_classic.memory import ConversationBufferMemory


memory = ConversationBufferMemory(return_messages = True)

C:\Users\pkmpp\AppData\Local\Temp\ipykernel_10684\1671511597.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages = True)


In [3]:
# There are several ways that we can add messages to our memory, using the save_context method we can add a user query (via the input key) and the AI's response (via the output key). So, to create the following conversation:

memory.save_context(
    {"input": "Hi, my name is James"},  # user message
    {"output": "Hey James, what's up? I'm an AI model called Zeta."},  # AI response
)
memory.save_context(
    {
        "input": "I'm researching the different types of conversational memory."
    },  # user message
    {"output": "That's interesting, what are some examples?"},  # AI response
)
memory.save_context(
    {
        "input": "I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory."
    },  # user message
    {"output": "That's interesting, what's the difference?"},  # AI response
)
memory.save_context(
    {
        "input": "Buffer memory just stores the entire conversation, right?"
    },  # user message
    {
        "output": "That makes sense, what about ConversationBufferWindowMemory?"
    },  # AI response
)
memory.save_context(
    {
        "input": "Buffer window memory stores the last k messages, dropping the rest."
    },  # user message
    {"output": "Very cool!"},  # AI response
)

In [4]:
memory.load_memory_variables({}) # Load history

{'history': [HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMess

In [5]:
# alternative method to add messages to out memory.
# With this other method, we pass individual user and AI messages via the add_user_message and add_ai_message methods. To reproduce what we did above, we do

memory = ConversationBufferMemory(return_messages=True)

memory.chat_memory.add_user_message("Hi, my name is James")
memory.chat_memory.add_ai_message("Hey James, what's up? I'm an AI model called Zeta.")
memory.chat_memory.add_user_message("I'm researching the different types of conversational memory.")
memory.chat_memory.add_ai_message("That's interesting, what are some examples?")
memory.chat_memory.add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
memory.chat_memory.add_ai_message("That's interesting, what's the difference?")
memory.chat_memory.add_user_message("Buffer memory just stores the entire conversation, right?")
memory.chat_memory.add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
memory.chat_memory.add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
memory.chat_memory.add_ai_message("Very cool!")

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMess

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = MODEL, base_url = BASE_URL)

In [7]:
from langchain_classic.chains import ConversationChain

chain = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)

C:\Users\pkmpp\AppData\Local\Temp\ipykernel_10684\1592351593.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use `langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chain = ConversationChain(


In [8]:
chain.invoke({"input":"What is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}), AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}), HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's the difference?", additional_kw

{'input': 'What is my name again?',
 'history': [HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}

#### `New Method`

ConversationBufferMemory with RunnableWithMessageHistory

In [ ]:
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate,
)

system_prompt = "You are a helpful assistant called Zeta"

prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt),
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("{query}")
    ]
)

pipeline = prompt_template | llm

In [11]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_map = {}
def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]

In [12]:
from langchain_core.runnables.history import RunnableWithMessageHistory

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",  # "query" need to match to the prompt_template
    history_messages_key="history",  # "history" need to match to the prompt_template
)

In [13]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"}, config={"session_id": "id_123"}
)

AIMessage(content='Hi James! It’s really nice to meet you. I’m Zeta, your helpful assistant. \n\nWhat can I do for you today? Do you have a question, need help with something, or just want to chat?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 27, 'total_tokens': 76, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemma3:4b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-539', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b60f2-3b55-76d3-b68f-af3aee460c75-0', usage_metadata={'input_tokens': 27, 'output_tokens': 49, 'total_tokens': 76, 'input_token_details': {}, 'output_token_details': {}})

In [14]:
pipeline_with_history.invoke(
    {"query": "What is my name again?"}, config={"session_id": "id_123"}
)

AIMessage(content='Your name is James! 😊 I just wanted to confirm – it’s always good to be sure. \n\nIs there anything you’d like to talk about, James?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 91, 'total_tokens': 128, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemma3:4b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-976', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b60f2-7cc3-72e2-87d0-9ad8794e96d5-0', usage_metadata={'input_tokens': 91, 'output_tokens': 37, 'total_tokens': 128, 'input_token_details': {}, 'output_token_details': {}})

In [15]:
pipeline_with_history.invoke(
    {"query": "I like Python coding language"}, config={"session_id": "id_123"}
)

AIMessage(content='That’s fantastic, James! Python is a really popular and versatile language. It’s known for being relatively easy to learn and it’s used in so many different areas – web development, data science, machine learning, scripting… \n\nWhat specifically do you enjoy about Python coding? Are you working on a particular project, or are you just interested in learning more?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 142, 'total_tokens': 219, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemma3:4b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-287', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b60f2-b87d-75c2-8d52-ac8c8c6c28b9-0', usage_metadata={'input_tokens': 142, 'output_tokens': 77, 'total_tokens': 219, 'input_token_details': {}, 'output_token_details': {}})

In [16]:
pipeline_with_history.invoke(
    {"query": "so i tell you that i like some coding language. can you repeat that to me what I like as a coding language."}, config={"session_id": "id_123"}
)

AIMessage(content='Okay, James, just to be sure I’ve got it right – you enjoy coding languages! Specifically, you mentioned you like Python. \n\nIs that accurate? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 253, 'total_tokens': 289, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemma3:4b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-607', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b60f3-525c-7770-ac71-5536f18dd166-0', usage_metadata={'input_tokens': 253, 'output_tokens': 36, 'total_tokens': 289, 'input_token_details': {}, 'output_token_details': {}})

In [17]:
pipeline_with_history.invoke(
    {"query": "yes that is correct."}, config={"session_id": "id_123"}
)

AIMessage(content='Great! It’s really cool that you’re into coding languages. Python is a fantastic choice. \n\nDo you have any specific questions about Python that you’d like to ask me, or would you like to talk about it a little more?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 303, 'total_tokens': 356, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemma3:4b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-578', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b60f3-e142-7690-ac8a-c394148b0cf5-0', usage_metadata={'input_tokens': 303, 'output_tokens': 53, 'total_tokens': 356, 'input_token_details': {}, 'output_token_details': {}})

## 2. ConversationBufferWindowMemory

In [1]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "none"
# MODEL = "functiongemma:270m" # still no good
# MODEL = "qwen3:0.6b"  # okay
MODEL = "gemma3:4b" # okay
BASE_URL = "http://localhost:11434/v1"

llm = ChatOpenAI(model=MODEL, base_url=BASE_URL)

In [2]:
from langchain_classic.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=4, return_messages=True)

C:\Users\pkmpp\AppData\Local\Temp\ipykernel_10172\392160499.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=4, return_messages=True)


In [3]:
memory.chat_memory.add_user_message("Hi, my name is James")
memory.chat_memory.add_ai_message("Hey James, what's up? I'm an AI model called Zeta.")
memory.chat_memory.add_user_message("I'm researching the different types of conversational memory.")
memory.chat_memory.add_ai_message("That's interesting, what are some examples?")
memory.chat_memory.add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
memory.chat_memory.add_ai_message("That's interesting, what's the difference?")
memory.chat_memory.add_user_message("Buffer memory just stores the entire conversation, right?")
memory.chat_memory.add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
memory.chat_memory.add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
memory.chat_memory.add_ai_message("Very cool!")

memory.load_memory_variables({})

{'history': [HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory stores the last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Very cool!', additional_kwargs={}, response_metadata={})]}

In [5]:
from langchain_classic.chains import ConversationChain

chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

C:\Users\pkmpp\AppData\Local\Temp\ipykernel_10172\1588115874.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use `langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chain = ConversationChain(


In [6]:
chain.invoke({"input": "What is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}), HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}), AIMessage(content='That makes sense, what about 

{'input': 'What is my name again?',
 'history': [HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory stores the last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Very cool!', additional_kw

#### `New Method`

ConversationBufferWindowMemory with RunnableWithMessageHistory

In [9]:
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate,
)

system_prompt = "You are a helpful assistant called Zeta"

prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt),
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("{query}")
    ]
)

pipeline = prompt_template | llm

In [10]:
from pydantic import BaseModel, Field
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage


class BufferWindowMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    k: int = Field(default_factory=int)

    def __init__(self, k: int):
        super().__init__(k=k)
        print(f"Initializing BufferWindowMessageHistory with k={k}")

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history, removing any messages beyond
        the last `k` messages.
        """
        self.messages.extend(messages)
        self.messages = self.messages[-self.k :]

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

In [11]:
chat_map = {}


def get_chat_history(session_id: str, k: int = 4) -> BufferWindowMessageHistory:
    print(f"get_chat_history called with session_id={session_id} and k={k}")
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = BufferWindowMessageHistory(k=k)
    # remove anything beyond the last
    return chat_map[session_id]

In [12]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="k",
            annotation=int,
            name="k",
            description="The number of messages to keep in the history",
            default=4,
        ),
    ],
)

In [13]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"configurable": {"session_id": "id_k4", "k": 4}},
)

get_chat_history called with session_id=id_k4 and k=4
Initializing BufferWindowMessageHistory with k=4


AIMessage(content='Hi James, it’s really nice to meet you! I’m Zeta, your helpful assistant. \n\nHow can I help you out today? Do you have a question, need some information, or just want to chat?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 27, 'total_tokens': 75, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemma3:4b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-266', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b653c-d2b2-77b1-adc9-aeba3c157082-0', usage_metadata={'input_tokens': 27, 'output_tokens': 48, 'total_tokens': 75, 'input_token_details': {}, 'output_token_details': {}})

In [16]:
chat_map["id_k4"]

BufferWindowMessageHistory(messages=[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi James, it’s really nice to meet you! I’m Zeta, your helpful assistant. \n\nHow can I help you out today? Do you have a question, need some information, or just want to chat?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 27, 'total_tokens': 75, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemma3:4b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-266', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b653c-d2b2-77b1-adc9-aeba3c157082-0', usage_metadata={'input_tokens': 27, 'output_tokens': 48, 'total_tokens': 75, 'input_token_details': {}, 'output_token_details': {}})], k=4)

In [17]:
chat_map["id_k4"].clear()  # clear the history

In [18]:
chat_map["id_k4"]

BufferWindowMessageHistory(messages=[], k=4)

In [19]:
# manually insert history
chat_map["id_k4"].add_user_message("Hi, my name is James")
chat_map["id_k4"].add_ai_message("I'm an AI model called Zeta.")
chat_map["id_k4"].add_user_message("I'm researching the different types of conversational memory.")
chat_map["id_k4"].add_ai_message("That's interesting, what are some examples?")
chat_map["id_k4"].add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
chat_map["id_k4"].add_ai_message("That's interesting, what's the difference?")
chat_map["id_k4"].add_user_message("Buffer memory just stores the entire conversation, right?")
chat_map["id_k4"].add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
chat_map["id_k4"].add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
chat_map["id_k4"].add_ai_message("Very cool!")

chat_map["id_k4"].messages

[HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Buffer window memory stores the last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Very cool!', additional_kwargs={}, response_metadata={})]

In [20]:
chat_map["id_k4"]

BufferWindowMessageHistory(messages=[HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}), AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Buffer window memory stores the last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}), AIMessage(content='Very cool!', additional_kwargs={}, response_metadata={})], k=4)

In [22]:
pipeline_with_history.invoke(
    {"query": "what is my name again?"},
    config={"configurable": {"session_id": "id_k4", "k": 4}}
)

get_chat_history called with session_id=id_k4 and k=4


AIMessage(content='You’re right to ask! My memory is a bit like a buffer window – it only holds the most recent information. \n\nYour name is Zeta. I’m still learning to keep track of things in the way humans do, so apologies for the confusion! 😊 \n\nWould you like to talk about something specific, or would you like me to refresh my memory about you?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 109, 'total_tokens': 188, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemma3:4b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-542', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b653f-9e21-75a0-a98f-14dae37651a0-0', usage_metadata={'input_tokens': 109, 'output_tokens': 79, 'total_tokens': 188, 'input_token_details': {}, 'output_token_details': {}})

In [23]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_k14", "k": 14}
)

get_chat_history called with session_id=id_k14 and k=14
Initializing BufferWindowMessageHistory with k=14


AIMessage(content="Hi James, it’s nice to meet you! I’m Zeta, your helpful assistant. \n\nHow can I assist you today? Do you have a question you'd like me to answer, or is there something you’d like me to help you do?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 27, 'total_tokens': 84, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemma3:4b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-550', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b6540-1ca4-76c2-8b9f-09b30785e5c4-0', usage_metadata={'input_tokens': 27, 'output_tokens': 57, 'total_tokens': 84, 'input_token_details': {}, 'output_token_details': {}})

In [24]:
chat_map["id_k14"].add_user_message("I'm researching the different types of conversational memory.")
chat_map["id_k14"].add_ai_message("That's interesting, what are some examples?")
chat_map["id_k14"].add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
chat_map["id_k14"].add_ai_message("That's interesting, what's the difference?")
chat_map["id_k14"].add_user_message("Buffer memory just stores the entire conversation, right?")
chat_map["id_k14"].add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
chat_map["id_k14"].add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
chat_map["id_k14"].add_ai_message("Very cool!")

chat_map["id_k14"].messages

[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hi James, it’s nice to meet you! I’m Zeta, your helpful assistant. \n\nHow can I assist you today? Do you have a question you'd like me to answer, or is there something you’d like me to help you do?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 27, 'total_tokens': 84, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemma3:4b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-550', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b6540-1ca4-76c2-8b9f-09b30785e5c4-0', usage_metadata={'input_tokens': 27, 'output_tokens': 57, 'total_tokens': 84, 'input_token_details': {}, 'output_token_details': {}}),
 HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
 AIMessa

In [ ]:
# new session

pipeline_with_history.invoke(
    {"query": "what is my name again?"},
    config={"session_id": "id_k14", "k": 14}
)

get_chat_history called with session_id=id_k14 and k=14


AIMessage(content="Your name is James! 😊 \n\nIt's great you’re digging into conversational memory – it’s a really fascinating area. Do you want to delve a little deeper into the differences between these memory types, or would you like me to explain how they relate to building a more sophisticated chatbot?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 223, 'total_tokens': 285, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemma3:4b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-937', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b6540-d114-7031-b58a-08bf5b9e7842-0', usage_metadata={'input_tokens': 223, 'output_tokens': 62, 'total_tokens': 285, 'input_token_details': {}, 'output_token_details': {}})

In [ ]:
chat_map["id_k14"].messages

[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hi James, it’s nice to meet you! I’m Zeta, your helpful assistant. \n\nHow can I assist you today? Do you have a question you'd like me to answer, or is there something you’d like me to help you do?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 27, 'total_tokens': 84, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemma3:4b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-550', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b6540-1ca4-76c2-8b9f-09b30785e5c4-0', usage_metadata={'input_tokens': 27, 'output_tokens': 57, 'total_tokens': 84, 'input_token_details': {}, 'output_token_details': {}}),
 HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
 AIMessa

: 

## 3. ConversationSummaryMemory

In [2]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "none"
# MODEL = "functiongemma:270m" # still no good
# MODEL = "qwen3:0.6b"  # okay
MODEL = "gemma3:4b" # okay
BASE_URL = "http://localhost:11434/v1"

llm = ChatOpenAI(model=MODEL, base_url=BASE_URL)

In [3]:
from langchain_classic.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm)

C:\Users\pkmpp\AppData\Local\Temp\ipykernel_6600\3945359647.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


In [4]:
from langchain_classic.chains import ConversationChain

chain = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

C:\Users\pkmpp\AppData\Local\Temp\ipykernel_6600\3667692610.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use `langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chain = ConversationChain(


In [6]:
chain.invoke({"input": "hello there my name is James"})
chain.invoke({"input": "I am researching the different types of conversational memory."})
chain.invoke({"input": "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory."})
chain.invoke({"input": "Buffer memory just stores the entire conversation"})
chain.invoke({"input": "Buffer window memory stores the last k messages, dropping the rest."})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hello there my name is James
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
James introduces himself, and Codex responds with a friendly greeting and reveals its identity as a large language model created by OpenAI. Codex then provides interesting information about the name "James," including its Hebrew origins and historical prevalence, demonstrating its extensive kn

{'input': 'Buffer window memory stores the last k messages, dropping the rest.',
 'history': 'James introduced Codex and its memory types, specifically ConversationBufferMemory and ConversationBufferWindowMemory, explaining them as part of “windowed memory” utilizing a sliding window to retain conversation context. ConversationBufferWindowMemory uses a fixed-size window for simplicity but limited retention, while ConversationBufferMemory employs decay functions to reduce the influence of older tokens, enabling longer context retention. Codex is open to further discussion on these functions and their relationship to the overall conversational AI architecture.',
 'response': 'You’ve hit on a really key aspect of how ConversationBufferWindowMemory works! It absolutely does store the last *k* messages – that’s the core of its design. The value of *k* determines the window size. Think of it like a sliding window that moves along the conversation as new messages arrive. When the window reach

In [7]:
chain.invoke({"input": "What is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
James introduced Codex and its memory types, focusing on “windowed memory” using ConversationBufferWindowMemory and ConversationBufferMemory. ConversationBufferWindowMemory maintains a sliding window of the last *k* messages – discarding older ones as new ones arrive – offering efficiency but limited retention. ConversationBufferMemory utilizes decay functions for longer context retention. The human clarified that *k* represents the window size, which is configurable and tied to the task and model capabilities, balancing context retention with computational cost. The AI offered to delve deeper into window size determination or strategies for selecting the optimal *k* v

{'input': 'What is my name again?',
 'history': 'James introduced Codex and its memory types, focusing on “windowed memory” using ConversationBufferWindowMemory and ConversationBufferMemory. ConversationBufferWindowMemory maintains a sliding window of the last *k* messages – discarding older ones as new ones arrive – offering efficiency but limited retention. ConversationBufferMemory utilizes decay functions for longer context retention. The human clarified that *k* represents the window size, which is configurable and tied to the task and model capabilities, balancing context retention with computational cost. The AI offered to delve deeper into window size determination or strategies for selecting the optimal *k* value.',
 'response': 'You’re James! I’ve got that noted in my current conversation context. Specifically, I recorded that you initiated the discussion about Codex and its memory types, focusing on windowed memory – particularly ConversationBufferWindowMemory and Conversatio

#### `New Method`

ConversationSummaryMemory with RunnableWithMessageHistory

In [ ]:
import os

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, SystemMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

os.environ["OPENAI_API_KEY"] = "none"
# MODEL = "functiongemma:270m" # still no good
# MODEL = "qwen3:0.6b"  # okay
MODEL = "gemma3:4b"  # okay
BASE_URL = "http://localhost:11434/v1"

llm = ChatOpenAI(model=MODEL, base_url=BASE_URL)

system_prompt = "You are a helpful assistant called Zeta"

prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt),
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("{query}"),
    ]
)

pipeline = prompt_template | llm


class ConversationSummaryMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    llm: ChatOpenAI = Field(default_factory=ChatOpenAI)

    def __init__(self, llm: ChatOpenAI):
        super().__init__(llm=llm)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history, removing any messages beyond
        the last `k` messages.
        """
        self.messages.extend(messages)
        # construct the summary chat messages
        summary_prompt = ChatPromptTemplate.from_messages(
            [
                SystemMessagePromptTemplate.from_template(
                    "Given the existing conversation summary and the new messages, "
                    "generate a new summary of the conversation. Ensuring to maintain "
                    "as much relevant information as possible."
                ),
                HumanMessagePromptTemplate.from_template(
                    "Existing conversation summary:\n{existing_summary}\n\n"
                    "New messages:\n{messages}"
                ),
            ]
        )
        # format the messages and invoke the LLM
        new_summary = self.llm.invoke(
            summary_prompt.format_messages(
                existing_summary=self.messages,
                messages=[x.content for x in messages],
            )
        )
        # replace the existing history with a single system summary message
        self.messages = [SystemMessage(content=new_summary.content)]

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []


chat_map = {}


def get_chat_history(
    session_id: str, llm: ChatOpenAI
) -> ConversationSummaryMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = ConversationSummaryMessageHistory(llm=llm)
    # return the chat history
    return chat_map[session_id]


pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=ChatOpenAI,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm,
        ),
    ],
)

In [2]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_123", "llm": llm}
)

AIMessage(content='Hi James! It’s nice to meet you. I’m Zeta. How’s your day going so far? Is there anything I can help you with today, or were you just saying hello?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 27, 'total_tokens': 70, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemma3:4b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-259', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b6b50-d59c-7fb3-a414-d00633a2455f-0', usage_metadata={'input_tokens': 27, 'output_tokens': 43, 'total_tokens': 70, 'input_token_details': {}, 'output_token_details': {}})

In [3]:
chat_map["id_123"].messages

[SystemMessage(content='Here’s a new summary of the conversation, incorporating the new messages:\n\nJames introduced himself and stated his name is James. Zeta responded warmly, introducing herself as Zeta and asking how James’ day was going, inquiring if she could assist him or if he was simply saying hello.', additional_kwargs={}, response_metadata={})]

In [4]:
pipeline_with_history.invoke(
    {"query": "I'm researching the different types of conversational memory."},
    config={"session_id": "id_123", "llm": llm},
)

chat_map["id_123"].messages

[SystemMessage(content='Okay, here’s a revised summary of the conversation, incorporating the new messages and maintaining the key information:\n\n**Summary:**\n\nThe conversation began with James introducing himself and expressing interest in researching the different types of conversational memory. Zeta responded by providing a detailed explanation of four key categories of conversational memory:\n\n*   **Episodic Memory (Within the Conversation):** Recollection of specific details from the conversation, similar to remembering a past event.\n*   **Priming (Implicit Conversation Memory):** Unconscious influence of previous conversations on responses.\n*   **Contextual Memory (Situational Awareness):** Remembering details about the situation surrounding the conversation.\n*   **Working Memory (Real-Time Processing):** Maintaining and manipulating information during the conversation.\n\nZeta then outlined the neural basis of conversational memory (prefrontal cortex, hippocampus, attenti

In [6]:
for msg in [
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest.",
]:
    pipeline_with_history.invoke(
        {"query": msg}, config={"session_id": "id_123", "llm": llm}
    )

In [7]:
chat_map["id_123"].messages

[SystemMessage(content='**Summary:**\n\nThe conversation continues to explore conversational memory, specifically focusing on “ConversationBufferWindowMemory.” Zeta explains that this memory type operates by storing the last ‘k’ messages of a conversation, discarding older ones. This ‘k’ value represents the window size – the amount of prior context the Large Language Model retains. It’s described as a “sliding window” that shifts forward as the conversation progresses. Zeta proposes discussing the implications of this approach, such as how the window size impacts coherence and its interaction with other conversational memory categories like Episodic, Priming, Contextual, and Working Memory.', additional_kwargs={}, response_metadata={})]

In [8]:
pipeline_with_history.invoke(
    {"query": "What is my name again?"}, config={"session_id": "id_123", "llm": llm}
)

AIMessage(content='You’ve been calling yourself Zeta! I’m here to help you, Zeta. 😊 \n\nDo you want me to remind you of anything else about our conversation so far?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 150, 'total_tokens': 188, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemma3:4b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-543', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b6b5f-f447-7a70-9cc1-7abee00c11ea-0', usage_metadata={'input_tokens': 150, 'output_tokens': 38, 'total_tokens': 188, 'input_token_details': {}, 'output_token_details': {}})

`video time -> 1:57:34`

## 4. ConversationSummaryBufferMemory

`not working. need a model of open ai`

In [ ]:
import os

from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "none"
# MODEL = "functiongemma:270m" # still no good
# MODEL = "qwen3:0.6b"  # okay
MODEL = "gemma3:4b" # okay
BASE_URL = "http://localhost:11434/v1"

llm = ChatOpenAI(model=MODEL, base_url=BASE_URL)

In [2]:
from langchain_classic.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=300,
    return_messages=True
)

C:\Users\pkmpp\AppData\Local\Temp\ipykernel_17220\114919256.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


In [3]:
from langchain_classic.chains import ConversationChain

chain = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)

C:\Users\pkmpp\AppData\Local\Temp\ipykernel_17220\1592351593.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use `langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chain = ConversationChain(


In [ ]:
chain.invoke({"input": "hi, my name is James"})

In [ ]:
for msg in [
    "I'm researching the different types of conversational memory.",
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest."
]:
    chain.invoke({"input": msg})

`new method`

ConversationSummaryBufferMemory with RunnableWithMessageHistory

`this part is working`

In [5]:
import os

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, SystemMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "none"
# MODEL = "functiongemma:270m" # still no good
# MODEL = "qwen3:0.6b"  # okay
MODEL = "gemma3:4b"  # okay
BASE_URL = "http://localhost:11434/v1"

llm = ChatOpenAI(model=MODEL, base_url=BASE_URL)

system_prompt = "You are a helpful assistant called Zeta"

prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt),
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("{query}"),
    ]
)

pipeline = prompt_template | llm


class ConversationSummaryBufferMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    llm: ChatOpenAI = Field(default_factory=ChatOpenAI)
    k: int = Field(default_factory=int)

    def __init__(self, llm: ChatOpenAI, k: int):
        super().__init__(llm=llm, k=k)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history, removing any messages beyond
        the last `k` messages and summarizing the messages that we
        drop.
        """
        existing_summary: SystemMessage | None = None
        old_messages: list[BaseMessage] | None = None
        # see if we already have a summary message
        if len(self.messages) > 0 and isinstance(self.messages[0], SystemMessage):
            print(">> Found existing summary")
            existing_summary = self.messages.pop(0)
        # add the new messages to the history
        self.messages.extend(messages)
        # check if we have too many messages
        if len(self.messages) > self.k:
            print(
                f">> Found {len(self.messages)} messages, dropping "
                f"oldest {len(self.messages) - self.k} messages."
            )
            # pull out the oldest messages...
            old_messages = self.messages[: -self.k]
            # ...and keep only the most recent messages
            self.messages = self.messages[-self.k :]
        if old_messages is None:
            print(">> No old messages to update summary with")
            # if we have no old_messages, we have nothing to update in summary
            return
        # construct the summary chat messages
        summary_prompt = ChatPromptTemplate.from_messages(
            [
                SystemMessagePromptTemplate.from_template(
                    "Given the existing conversation summary and the new messages, "
                    "generate a new summary of the conversation. Ensuring to maintain "
                    "as much relevant information as possible."
                ),
                HumanMessagePromptTemplate.from_template(
                    "Existing conversation summary:\n{existing_summary}\n\n"
                    "New messages:\n{old_messages}"
                ),
            ]
        )
        # format the messages and invoke the LLM
        new_summary = self.llm.invoke(
            summary_prompt.format_messages(
                existing_summary=existing_summary, old_messages=old_messages
            )
        )
        print(f">> New summary: {new_summary.content}")
        # prepend the new summary to the history
        self.messages = [SystemMessage(content=new_summary.content)] + self.messages

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []


chat_map = {}


def get_chat_history(
    session_id: str, llm: ChatOpenAI, k: int
) -> ConversationSummaryBufferMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = ConversationSummaryBufferMessageHistory(llm=llm, k=k)
    # return the chat history
    return chat_map[session_id]


pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=ChatOpenAI,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm,
        ),
        ConfigurableFieldSpec(
            id="k",
            annotation=int,
            name="k",
            description="The number of messages to keep in the history",
            default=4,
        ),
    ],
)

In [6]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_123", "llm": llm, "k": 4}
)
chat_map["id_123"].messages

>> No old messages to update summary with


[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hi James, it’s nice to meet you! I’m Zeta, your helpful assistant. \n\nHow can I assist you today? Do you have something specific you'd like to talk about, or were you just saying hello?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 27, 'total_tokens': 77, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemma3:4b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-47', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b7023-5acc-7df3-bf3c-7ec939a9485d-0', usage_metadata={'input_tokens': 27, 'output_tokens': 50, 'total_tokens': 77, 'input_token_details': {}, 'output_token_details': {}})]

In [7]:
for i, msg in enumerate(
    [
        "I'm researching the different types of conversational memory.",
        "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
        "Buffer memory just stores the entire conversation",
        "Buffer window memory stores the last k messages, dropping the rest.",
    ]
):
    print(f"---\nMessage {i+1}\n---\n")
    pipeline_with_history.invoke(
        {"query": msg}, config={"session_id": "id_123", "llm": llm, "k": 4}
    )

---
Message 1
---

>> No old messages to update summary with
---
Message 2
---

>> Found 6 messages, dropping oldest 2 messages.
>> New summary: James introduced himself and is now speaking with Zeta, a helpful assistant. Zeta greeted him and asked how she could assist him, opening the conversation for a specific request or simply a chat.
---
Message 3
---

>> Found existing summary
>> Found 6 messages, dropping oldest 2 messages.
>> New summary: James is researching the different types of conversational memory, a fascinating area of psychology. Zeta is assisting him by exploring his specific interests within this topic. She’s offered several starting points for discussion, including models of conversational memory (like the Interactive Activation Model and Situation Model), types of information remembered in conversation (topic, speaker identity, context), and the cognitive processes involved (attention, working memory, semantic memory). Zeta is now asking James to indicate where he’d

---

## Agents

In [1]:
# %pip install google-search-results

In [ ]:
from langchain_core.tools import tool


@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

# or it can be like this `def add(x: float, y: float, z: float | None = 0.3)`

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y


@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x**y


@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

In [2]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x000001E5FABDC900>)

In [3]:
print(f"{add.name =}\n{add.description =}")

add.name ='add'
add.description ="Add 'x' and 'y'."


In [4]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [7]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'x' to the power of 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

In [5]:
# When invoking the tool, a JSON string output by the LLM will be parsed into JSON and then consumed as kwargs, similar to the below:
import json

llm_output_string = '{"x": 5, "y": 2}'  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

In [6]:
# This is then passed into the tool function as kwargs (keyword arguments) as indicated by the ** operator - the ** operator is used to unpack the dictionary into keyword arguments.
exponentiate.func(**llm_output_dict)

25

In [8]:
add.func(**llm_output_dict)

7

`video time : 2:16:40`

### Creating an Agent